In [1]:
import numpy as np
import pandas as pd

Let us suppose that we have a dataframe with 5 rows. Each row gives us the values of a quantities in each of 5 bins. So, the columns are numbers, means of some quantity X and standard deviations of X. We will assume that in each of the bins, X is drawn from a Normal distribution, and so characterizing the mean and the standard deviation in each bin characterizes this distribution.

Q: How do we create a set of random samples of X so, that in each bin the values are drawn from the appropriate Normal distribution as described above, but the total number of values is an input Nobjs.

We will generate a toy table for this first: 

In [2]:
# Code to generate the toy example (let us not worry how this code works)
nums = np.arange(1000, 6000, 1000) \
    + np.round(np.random.RandomState(0).normal(0., 200., size=5,)).astype(np.int)
df = pd.DataFrame(dict(Numbers=nums, meanX=np.power(nums, 0.5)/5., 
                       stdX=np.power(nums, 0.1)))

In [3]:
df

,Numbers,meanX,stdX
0,1353,7.356630,2.056505
1,2080,9.121403,2.146873
2,3196,11.306635,2.241097
3,4448,13.338666,2.316416
4,5374,14.661514,2.360640


## Step 1
To answer the question: we will assume that the numbers in each bin relative to the total 
    stay the same when we change the total numbers. So first :

In [4]:
df['frequencies'] = df.Numbers / df.Numbers.sum()

In [5]:
df

,Numbers,meanX,stdX,frequencies
0,1353,7.356630,2.056505,0.082244
1,2080,9.121403,2.146873,0.126436
2,3196,11.306635,2.241097,0.194274
3,4448,13.338666,2.316416,0.270379
4,5374,14.661514,2.360640,0.326667


## Step 2

Now, given a total number of objects Nobj, we can find approimately how many we expect in each bin = Nobj * frequencies (but that might be a fraction, so we will round it to an integer). Let us try Nobj = 50000

In [6]:
numObjectsPerBin = np.round(df.frequencies * 50000).astype(np.int)
print(numObjectsPerBin)

0     4112
1     6322
2     9714
3    13519
4    16333
Name: frequencies, dtype: int64


We can check (as it obviously must) that this matches our numbers if Nobj equals the total number of objects in our 
toy example:

In [7]:
np.round(df.frequencies * df.Numbers.sum()).astype(np.int)

0    1353
1    2080
2    3196
3    4448
4    5374
Name: frequencies, dtype: int64

## Step 3 Now for each bin we mgith want to draw numbers from a normal distribution with size = number of objects in that bin

For each bin this is now easy: (some syntax about how to access the ith row and two columns by name from a dataframe is necessary, but fairly intuitive)

In [8]:
m, s  = df.ix[0, ['meanX', 'stdX']] # Now the mean of the 0th bin is assigned to m, and std to s
X = np.random.normal(m, s, size=numObjectsPerBin.ix[0] )
print(X)

[ 11.82148782   7.87717186  10.39761086 ...,   4.87166367   5.01330949
   7.75538296]


So, what we need to do is loop through the bins, and keep appending X to some list

In [9]:
XVals = []
for i in range(len(df)):
    m, s  = df.ix[i, ['meanX', 'stdX']]
    # We will convert the numpy array to list, but that may not be necessary
    X = np.random.normal(m, s, size=numObjectsPerBin.ix[i]).tolist()
    XVals.append(X)


In [10]:
XVals

[[5.977929317321006,
  7.415968145437815,
  9.593962865036984,
  8.69288999954274,
  7.235988149050986,
  9.317371778308189,
  5.660328881739261,
  5.286273139466873,
  9.68704747052957,
  5.620947551989206,
  10.999003566018434,
  10.88592640858787,
  1.2100675371907483,
  9.913117052748271,
  4.494208961387583,
  8.85548287771135,
  8.01679558684271,
  6.52528470115363,
  10.313444731925701,
  7.8456216436375925,
  7.604878483690177,
  4.180954591764744,
  9.510805240368942,
  8.113457111496306,
  8.503954933487032,
  8.410968475562129,
  5.520801380779843,
  4.480321566129454,
  6.847019614730639,
  9.227207588416197,
  5.299435069154212,
  9.283007562680158,
  9.344162387451298,
  5.311724129980107,
  8.621102502438086,
  6.710087399957756,
  8.817064581548362,
  10.514411609404842,
  11.449080605992766,
  7.690103992145982,
  5.958763098636295,
  11.549395866196829,
  7.009285899017684,
  4.3812605904467485,
  7.891458767344362,
  9.226088778441664,
  1.3182701545906061,
  6.56684

XVals is a list of lists. The 0th element of XVals is a list which has all of the Xs sampled 
in the 0th bin and likewise.

## We can check the frequencies match up

`map` is a useful function to know about (but not essential, you can use for loops or preferably us list comprehensions in its place). But here is what it does:
    sequence = [a, b, c] 
    map( func, sequence) = list(func(a), func(b), func(c)) 

    So we can use this to find the frequencies and also check that the total adds up to Nobj (approximately)

In [11]:
np.array(map(len, XVals)) / np.float(sum(map(len, XVals)))

array([ 0.08224,  0.12644,  0.19428,  0.27038,  0.32666])

In [12]:
totalobjs = sum(map(len, XVals))

And we can find their means and std deviations

In [13]:
map(np.mean, XVals)

[7.3054546438422134,
 9.1108221778337786,
 11.343646447755241,
 13.3128122122652,
 14.669735561599337]

In [14]:
map(np.std, XVals)

[2.0383901924493655,
 2.1392975839100177,
 2.2287632504115118,
 2.3305806587404332,
 2.3679599054627305]

## Randomness and Reproducibility

There was a question about getting different values from each random draw. As guessed, this is expected, but there are times when you want to get the same answer to be able to reproduce older calculations. One of the ways to achieve this is by supplying a seed

In [15]:
seed = 1
rng = np.random.RandomState(seed)
rng.normal(0, 1, size= 50)

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
       -2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038,
        1.46210794, -2.06014071, -0.3224172 , -0.38405435,  1.13376944,
       -1.09989127, -0.17242821, -0.87785842,  0.04221375,  0.58281521,
       -1.10061918,  1.14472371,  0.90159072,  0.50249434,  0.90085595,
       -0.68372786, -0.12289023, -0.93576943, -0.26788808,  0.53035547,
       -0.69166075, -0.39675353, -0.6871727 , -0.84520564, -0.67124613,
       -0.0126646 , -1.11731035,  0.2344157 ,  1.65980218,  0.74204416,
       -0.19183555, -0.88762896, -0.74715829,  1.6924546 ,  0.05080775,
       -0.63699565,  0.19091548,  2.10025514,  0.12015895,  0.61720311])

The next 50 will be different 

In [16]:
rng.normal(0, 1, size=50)

array([ 0.30017032, -0.35224985, -1.1425182 , -0.34934272, -0.20889423,
        0.58662319,  0.83898341,  0.93110208,  0.28558733,  0.88514116,
       -0.75439794,  1.25286816,  0.51292982, -0.29809284,  0.48851815,
       -0.07557171,  1.13162939,  1.51981682,  2.18557541, -1.39649634,
       -1.44411381, -0.50446586,  0.16003707,  0.87616892,  0.31563495,
       -2.02220122, -0.30620401,  0.82797464,  0.23009474,  0.76201118,
       -0.22232814, -0.20075807,  0.18656139,  0.41005165,  0.19829972,
        0.11900865, -0.67066229,  0.37756379,  0.12182127,  1.12948391,
        1.19891788,  0.18515642, -0.37528495, -0.63873041,  0.42349435,
        0.07734007, -0.34385368,  0.04359686, -0.62000084,  0.69803203])

In [17]:
# But if you want to reproduce the first 50, you can do so by using the same seed
rng = np.random.RandomState(seed)
rng.normal(0,1, size=60)

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
       -2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038,
        1.46210794, -2.06014071, -0.3224172 , -0.38405435,  1.13376944,
       -1.09989127, -0.17242821, -0.87785842,  0.04221375,  0.58281521,
       -1.10061918,  1.14472371,  0.90159072,  0.50249434,  0.90085595,
       -0.68372786, -0.12289023, -0.93576943, -0.26788808,  0.53035547,
       -0.69166075, -0.39675353, -0.6871727 , -0.84520564, -0.67124613,
       -0.0126646 , -1.11731035,  0.2344157 ,  1.65980218,  0.74204416,
       -0.19183555, -0.88762896, -0.74715829,  1.6924546 ,  0.05080775,
       -0.63699565,  0.19091548,  2.10025514,  0.12015895,  0.61720311,
        0.30017032, -0.35224985, -1.1425182 , -0.34934272, -0.20889423,
        0.58662319,  0.83898341,  0.93110208,  0.28558733,  0.88514116])

What you shoudl be able to see in this 60 is that the first 50 are the first 50 that we had previously. The next 10 are actually the first 10 from the second fifty we tried.